<a href="https://colab.research.google.com/github/freddyduitama/GVD/blob/master/0_4_dataframes_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuración plataforma

In [0]:
# instala el ambiente de spark..solo se corre una vez
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
#Configura variables de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [0]:
#import 
import findspark
findspark.init()
from pyspark.sql import SparkSession , SQLContext
from pyspark import SparkContext,SparkConf

In [0]:
# monta el google drive para usar sus archivos
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [0]:
!ls -l /gdrive/'My Drive'/'Colab Notebooks/Data'

In [0]:
# definir la sesion. Note que Inicia con pyspark 
conf = SparkConf().setAppName("ejemplo").setMaster("local[*]")
sc = SparkContext(conf=conf)
spark = SQLContext(sc)

In [0]:
#inicia sesion en el cluster. Note que inicia con pyspark.sql
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Ejemplo 1: Lee archivo txt , crea tipo Row , crea DataFrame y crea vista temporal para usar con SQL

In [0]:
#  INST A: Opcional..si quiere subir archivos al ambiente de trabajo
from google.colab import files
datafile = files.upload()

In [0]:
from pyspark import Row
lines = sc.textFile("/gdrive/My Drive/Colab Notebooks/Data/valores.txt")

In [0]:
lines.take(3)

['Juan 20', 'Pedro 23', 'Maria 27']

In [0]:
parts = lines.map( lambda s  : s.split() )

In [0]:
parts.take(3)

[['Juan', '20'], ['Pedro', '23'], ['Maria', '27']]

In [0]:
people = parts.map(lambda p : Row(name=p[0], age=p[1]))

In [0]:
people.take(3)

[Row(age='20', name='Juan'),
 Row(age='23', name='Pedro'),
 Row(age='27', name='Maria')]

In [0]:
dfpeople = spark.createDataFrame(people)

In [0]:
dfpeople.createOrReplaceTempView("people")

In [0]:
teenagers = spark.sql("SELECT name , age FROM people WHERE age > 21 AND age < 29")

In [0]:
teenagers.show()

+-----+---+
| name|age|
+-----+---+
|Pedro| 23|
|Maria| 27|
+-----+---+



# Crea Dataframe desde archivo txt y luego crea vista temporal para consultas SQL

In [0]:
# importa librerias para manejar esquemas y tipos de datos con dataframe
from pyspark.sql.types import StructField, StructType, StringType, LongType

In [0]:
# define esquema a ser usado por el dataframe
myManualSchema = StructType([
  StructField("DEST_COUNTRY_NAME", StringType(), True),
  StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
  StructField("count", LongType(), False, metadata={"hello":"world"})
])

In [0]:
# lee archivo usando esquema previamente definido
df = spark.read.format("csv").schema(myManualSchema).load("/gdrive/My Drive/Colab Notebooks/Data/2015-summary.csv")

In [0]:
df.printSchema

In [0]:
df.createOrReplaceTempView("flights")

In [0]:
sqlWay = spark.sql(" SELECT DEST_COUNTRY_NAME, sum(count)  as  Total FROM flights GROUP BY DEST_COUNTRY_NAME ORDER BY Total DESC")

In [0]:
sqlWay.show()

+------------------+------+
| DEST_COUNTRY_NAME| Total|
+------------------+------+
|     United States|411293|
|            Canada|  8399|
|            Mexico|  7140|
|    United Kingdom|  2025|
|             Japan|  1548|
|           Germany|  1468|
|Dominican Republic|  1353|
|       South Korea|  1048|
|       The Bahamas|   955|
|            France|   935|
|          Colombia|   873|
|            Brazil|   853|
|       Netherlands|   776|
|             China|   772|
|           Jamaica|   666|
|        Costa Rica|   588|
|       El Salvador|   561|
|            Panama|   510|
|              Cuba|   466|
|             Spain|   420|
+------------------+------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import desc
dfOutput = df.groupBy("DEST_COUNTRY_NAME").sum("count").withColumnRenamed("sum(count)", "total").sort(desc("total")).show()

+------------------+------+
| DEST_COUNTRY_NAME| total|
+------------------+------+
|     United States|411293|
|            Canada|  8399|
|            Mexico|  7140|
|    United Kingdom|  2025|
|             Japan|  1548|
|           Germany|  1468|
|Dominican Republic|  1353|
|       South Korea|  1048|
|       The Bahamas|   955|
|            France|   935|
|          Colombia|   873|
|            Brazil|   853|
|       Netherlands|   776|
|             China|   772|
|           Jamaica|   666|
|        Costa Rica|   588|
|       El Salvador|   561|
|            Panama|   510|
|              Cuba|   466|
|             Spain|   420|
+------------------+------+
only showing top 20 rows



In [0]:
!cat /gdrive/'My Drive'/'Colab Notebooks'/Data/2015-summary.csv